In [150]:
%matplotlib inline

import os
import cv2 as cv
import numpy as np
from PIL import Image

In [151]:
def read_frames(vidcap, max_frame):
    success, frame, frames = True, None, []
    while success and len(frames) < max_frame:
        success, frame = vidcap.read()
        frames.append(cv.cvtColor(frame, cv.COLOR_BGR2RGB))
    return success, frames

In [152]:
def skip_frames(frames, sample_interval):
    use_frames = []
    for i, frame in enumerate(frames):
        if i % sample_interval == 0:
            use_frames.append(frame)
    return use_frames

In [153]:
def output_frames(frames, location, start_id):
    for i, frame in enumerate(frames):
        filename = os.path.join(location, '{id}.png'.format(id=start_id + i))
        Image.fromarray(frame).save(filename)
    print('Done. Saved {cnt} images.'.format(cnt=len(frames)))

In [154]:
def downscale_frames(frames, target_size):
    downscaled_frames = []
    for frame in frames:
        downscaled_frames.append(np.array(Image.fromarray(frame).resize(target_size)))
    return downscaled_frames

In [155]:
def generate_frames(vidcap, sample_interval, val_rate, output_dir, start_id, downscale_res):
    success = True
    total_samples = 0
    hi_res_train_location = os.path.join(output_dir, 'hi_res_train')
    hi_res_val_location = os.path.join(output_dir, 'hi_res_val')
    low_res_train_location = os.path.join(output_dir, 'low_res_train')
    low_res_val_location = os.path.join(output_dir, 'low_res_val')
    while success:
        print('Generate sample #{id}'.format(id=total_samples))
        success, frames = read_frames(vidcap, max_frame=clip_frame_cnt)
        use_frames = skip_frames(frames, sample_interval)
        np.random.shuffle(use_frames)
        val_cnt = int(len(use_frames) * val_rate)
        hi_res_train_frames = use_frames[val_cnt:]
        hi_res_val_frames = use_frames[:val_cnt]
        low_res_train_frames = downscale_frames(hi_res_train_frames, downscale_res)
        low_res_val_frames = downscale_frames(hi_res_val_frames, downscale_res)
        output_frames(hi_res_train_frames, hi_res_train_location, total_samples + start_id)
        output_frames(hi_res_val_frames, hi_res_val_location, total_samples + start_id)
        output_frames(low_res_train_frames, low_res_train_location, total_samples + start_id)
        output_frames(low_res_val_frames, low_res_val_location, total_samples + start_id)
        total_samples += len(use_frames)
    print('Done.')

In [156]:
clip_frame_cnt = 1000
val_rate = 0.2
original_width = 1278
original_height = 720
original_res = (original_width, original_height)
downscale_res = (original_width // 2, original_height // 2)
frame_sample_interval = 5
hi_res_video = 'high_res_0.mp4'
low_res_video = 'sample.mp4'
data_dir = os.path.join('..', 'data')

In [157]:
hi_res_vidcap = cv.VideoCapture(os.path.join(data_dir, hi_res_video))
low_res_vidcap = cv.VideoCapture(os.path.join(data_dir, low_res_video))

In [158]:
generate_frames(
    vidcap=hi_res_vidcap,
    sample_interval=frame_sample_interval,
    val_rate=val_rate,
    start_id = 0,
    output_dir=os.path.join(data_dir, 'cba'),
    downscale_res=downscale_res
)

Generate sample #0
Done. Saved 160 images.
Done. Saved 40 images.
Done. Saved 160 images.
Done. Saved 40 images.
Generate sample #200
Done. Saved 160 images.
Done. Saved 40 images.
Done. Saved 160 images.
Done. Saved 40 images.
Generate sample #400
Done. Saved 160 images.
Done. Saved 40 images.
Done. Saved 160 images.
Done. Saved 40 images.
Generate sample #600
Done. Saved 160 images.
Done. Saved 40 images.
Done. Saved 160 images.
Done. Saved 40 images.
Generate sample #800
Done. Saved 160 images.
Done. Saved 40 images.
Done. Saved 160 images.
Done. Saved 40 images.
Generate sample #1000
Done. Saved 160 images.
Done. Saved 40 images.
Done. Saved 160 images.
Done. Saved 40 images.
Generate sample #1200
Done. Saved 160 images.
Done. Saved 40 images.
Done. Saved 160 images.
Done. Saved 40 images.
Generate sample #1400
Done. Saved 160 images.
Done. Saved 40 images.
Done. Saved 160 images.
Done. Saved 40 images.
Generate sample #1600
Done. Saved 160 images.
Done. Saved 40 images.
Done. Save

error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-kh7iq4w7\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
hi_res_vidcap.release()
low_res_vidcap.release()